In [1]:
import sys
import pandas as pd

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
import os
import codecs

rexa_dir = '/home/is2548/ML/entity-resolution/data/'

field_names = [
    'author-in-focus', 'author-in-focus-score', 'authorlist', 'title',
    'alt-authorlist', 'journal', 'abstract', 'body', 'institution',
    'email', 'altTitle', 'keyword', 'editor', 'year'
]

authors_f = []

for root, dirs, files in os.walk(rexa_dir, topdown=False):
    for f in files:
        if f.endswith('.txt'):
            authors_f.append('{}/{}'.format(root, f))

In [3]:
def raw2dict(raw):
    return { entry.split(':', 1)[0].strip():entry.split(':', 1)[1].strip() for entry in raw if entry.strip() != ''}

In [20]:
from xml.etree import ElementTree as ET

def clean_name(xml_string):
    root = ET.fromstring(xml_string)
    return ' '.join([child.text for child in root])

In [5]:
authors_d = []

for file_path in authors_f:
    with codecs.open(file_path, 'r', 'utf-8', errors='ignore') as f:
        authors_d.append(raw2dict(f.readlines()))
print(len(authors_d))

3002


In [6]:
for author_d in authors_d:
    author_d['author-in-focus'] = clean_name(author_d['author-in-focus'])
    author_d['authorlist'] = ', '.join([clean_name(author_name) for author_name in author_d['authorlist'].split('%%')])
    if author_d.get('alt-authorlist', None) is not None:
        author_d['alt-authorlist'] = ', '.join(clean_name(author_name) for author_name in author_d['alt-authorlist'].split('%%'))
    if author_d.get('keyword', None) is not None:
        author_d['keyword'] = ', '.join([keyword.strip() for keyword in author_d['keyword'].split(',') ])
    if author_d.get('author-in-focus-score', None) is not None:
        author_d['author-in-focus-score'] = float(author_d['author-in-focus-score'])
print(authors_d[0])

{u'author-in-focus': 'Marilyn Jordan', u'abstract': u'Productivity of temperate forests typically is limited by nitrogen supply Nitrogen cycling in forests is linked to regional and global environmental issues, such as groundwater quality and the trace gases composition of the atmosphere. Forests are being fertilized to stimulate forest growth to meet increasing demands for forest products, and are being used as "living filters" to remove nitrogen from applied wastewater. Extensive forested areas are receiving excessive additions of nitrogen through nitrogenous components of acid rain. Although general patterns of nitrogen cycling in unimpacted forests are known, little is known about the nitrogen accumulation potential of forests, or the ability of forests to handle chronic excess nitrogen additions. This research is first of all a comparative ecosystem study of the effects of excess nitrogen additions on nitrogen cycling in early, mid- and late-successional forests on Cape Cod, Massa

In [7]:
def fill_na(author):
    for field in field_names:
        if author.get(field, None) is None:
            author[field] = None
    return author

In [8]:
# # Clean author data
# authors_dict = {}

# for index, author_d in enumerate(authors_d):
#     fill_na(author_d)
#     author_d['author-in-focus'] = clean_name(author_d['author-in-focus'])
#     author_d['authorlist'] = tuple([clean_name(author_name) for author_name in author_d['authorlist'].split('%%')])
#     if author_d.get('alt-authorlist', None) is not None:
#         author_d['alt-authorlist'] = tuple(clean_name(author_name) for author_name in author_d['alt-authorlist'].split('%%'))
#     if author_d.get('keyword', None) is not None:
#         author_d['keyword'] = tuple([keyword.strip() for keyword in author_d['keyword'].split(',') ])
#     if author_d.get('author-in-focus-score', None) is not None:
#         author_d['author-in-focus-score'] = float(author_d['author-in-focus-score'])
#     authors_dict[index] = author_d

In [9]:
# print(authors_dict.pop(1))

In [10]:
# import dedupe

# fields = [
#     {'field' : 'author-in-focus', 'type': 'String'},
#     {'field' : 'author-in-focus-score', 'type': 'Price'},
#     {'field' : 'authorlist', 'type': 'Set'},
#     {'field' : 'alt-authorlist', 'type': 'Set', 'has missing' : True},
#     {'field' : 'email', 'type': 'String', 'has missing' : True},
#     {'field' : 'keyword', 'type': 'Set', 'has missing' : True},
#     {'field' : 'abstract', 'type': 'Text', 'has missing' : True},
#     {'field' : 'body', 'type': 'Text', 'has missing' : True},
#     {'field' : 'journal', 'type': 'String', 'has missing' : True},
#     {'field' : 'institution', 'type': 'String', 'has missing' : True},
# ]

# deduper = dedupe.Dedupe(fields)

# deduper.sample(authors_dict)

# dedupe.consoleLabel(deduper)

# deduper.train()

# training_file = 'author_training.json'

# with open(training_file, 'w') as tf:
#     deduper.writeTraining(tf)
    
# threshold = deduper.threshold(authors_dict, recall_weight=1)

# print('clustering...')
# clustered_dupes = deduper.match(authors_dict, threshold)

# print('# duplicate sets', len(clustered_dupes))

In [11]:
authors_df = pd.DataFrame.from_dict(authors_d)
authors_df = authors_df[field_names]
authors_df.describe()

,author-in-focus-score
count,3002.000000
mean,0.925739
std,0.112081
min,0.507084
25%,0.927770
50%,0.975576
75%,1.000000
max,1.000000


In [12]:
authors_df.head()

,author-in-focus,author-in-focus-score,authorlist,title,alt-authorlist,journal,abstract,body,institution,email,altTitle,keyword,editor,year
0,Marilyn Jordan,0.944252,"Marilyn Jordan,",ROW: Nitrogen Cycling in Forests: A Macroplot ...,NaN,NaN,Productivity of temperate forests typically is...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M A Jordan,0.927770,"W B Derry, L Wilson, M A Jordan",Substoichiometric binding of taxol suppresses ...,"W B Derry, L Wilson, M A Jordan",Biochemistry.year: 1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M A Jordan,0.927770,"M A Jordan, K Wendell, S Gardiner, W B Derry, ...",Mitotic block induced in HeLa cells by low con...,"M A Jordan, K Wendell, S Gardiner, W B Derry, ...",Cancer Res.year: 1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M Jordan,0.944252,"E Peterson, M Woods, S Dmowski, G Derimanov, M...",Coupling of the TCR to integrin activation by ...,"E Peterson, M Woods, S Dmowski, G Derimanov, M...","Science,year: 2001",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Michael P Jordan,0.927770,Michael P Jordan,Rhetoric of Everyday English Texts,Michael P Jordan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
from csv import DictWriter

with open('authors.csv', 'w') as authors_file:
    writer = DictWriter(authors_file, field_names)
    writer.writeheader()
    for row in authors_d:
        writer.writerow(row)

In [14]:
df = pd.read_csv('./authors.csv')
df.describe()

,author-in-focus-score,year
count,3002.000000,65.000000
mean,0.925739,1997.738462
std,0.112081,4.338335
min,0.507084,1983.000000
25%,0.927770,1995.000000
50%,0.975576,1998.000000
75%,1.000000,2001.000000
max,1.000000,2004.000000


In [15]:
df.head()

,author-in-focus,author-in-focus-score,authorlist,title,alt-authorlist,journal,abstract,body,institution,email,altTitle,keyword,editor,year
0,Marilyn Jordan,0.944252,"Marilyn Jordan,",ROW: Nitrogen Cycling in Forests: A Macroplot ...,NaN,NaN,Productivity of temperate forests typically is...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M A Jordan,0.927770,"W B Derry, L Wilson, M A Jordan",Substoichiometric binding of taxol suppresses ...,"W B Derry, L Wilson, M A Jordan",Biochemistry.year: 1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M A Jordan,0.927770,"M A Jordan, K Wendell, S Gardiner, W B Derry, ...",Mitotic block induced in HeLa cells by low con...,"M A Jordan, K Wendell, S Gardiner, W B Derry, ...",Cancer Res.year: 1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M Jordan,0.944252,"E Peterson, M Woods, S Dmowski, G Derimanov, M...",Coupling of the TCR to integrin activation by ...,"E Peterson, M Woods, S Dmowski, G Derimanov, M...","Science,year: 2001",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Michael P Jordan,0.927770,Michael P Jordan,Rhetoric of Everyday English Texts,Michael P Jordan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
len(df)

3002

In [16]:
col_missing = []
for col in df.columns.tolist():
    col_missing.append((col, df[col].isnull().sum() * 1. / len(df)))
s = sorted(col_missing, key=lambda x: x[1])
[x[0] for x in s]

['author-in-focus',
 'author-in-focus-score',
 'authorlist',
 'title',
 'alt-authorlist',
 'journal',
 'abstract',
 'body',
 'institution',
 'email',
 'altTitle',
 'keyword',
 'editor',
 'year']

In [17]:
import jellyfish

In [84]:
pairwise_df = pd.DataFrame(
    columns=[
        'damerau_levenshtein_distance', 'hamming_distance', 'jaro_distance',
        'jaro_winkler', 'levenshtein_distance', 'match_rating_comparison'
    ]
)

In [97]:
def creat_row(df, row_1, row_2):
    row0 = df.ix[row_1]
    row1 = df.ix[row_2]


    aio0 = row0['author-in-focus']
    aio1 = row1['author-in-focus']
    return [
        jellyfish.damerau_levenshtein_distance(aio0.decode('unicode-escape'), aio1.decode('unicode-escape')),
        jellyfish.hamming_distance(aio0.decode('unicode-escape'), aio1.decode('unicode-escape')),
        jellyfish.jaro_distance(aio0.decode('unicode-escape'), aio1.decode('unicode-escape')),
        jellyfish.jaro_winkler(aio0.decode('unicode-escape'), aio1.decode('unicode-escape')),
        jellyfish.levenshtein_distance(aio0.decode('unicode-escape'), aio1.decode('unicode-escape')),
        jellyfish.match_rating_comparison(aio0.decode('unicode-escape'), aio1.decode('unicode-escape'))
    ]

In [100]:
for n2 in range(4):
    print creat_row(df, 0, n2)

[0, 0, 1.0, 1.0, 0, True]
[6, 13, 0.6654761904761904, 0.6654761904761904, 6, True]
[6, 13, 0.6654761904761904, 0.6654761904761904, 6, True]
[6, 13, 0.7321428571428571, 0.7589285714285714, 6, True]


In [99]:
df.head()

,author-in-focus,author-in-focus-score,authorlist,title,alt-authorlist,journal,abstract,body,institution,email,altTitle,keyword,editor,year
0,Marilyn Jordan,0.944252,"Marilyn Jordan,",ROW: Nitrogen Cycling in Forests: A Macroplot ...,NaN,NaN,Productivity of temperate forests typically is...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M A Jordan,0.927770,"W B Derry, L Wilson, M A Jordan",Substoichiometric binding of taxol suppresses ...,"W B Derry, L Wilson, M A Jordan",Biochemistry.year: 1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M A Jordan,0.927770,"M A Jordan, K Wendell, S Gardiner, W B Derry, ...",Mitotic block induced in HeLa cells by low con...,"M A Jordan, K Wendell, S Gardiner, W B Derry, ...",Cancer Res.year: 1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M Jordan,0.944252,"E Peterson, M Woods, S Dmowski, G Derimanov, M...",Coupling of the TCR to integrin activation by ...,"E Peterson, M Woods, S Dmowski, G Derimanov, M...","Science,year: 2001",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Michael P Jordan,0.927770,Michael P Jordan,Rhetoric of Everyday English Texts,Michael P Jordan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
pairwise_df

,damerau_levenshtein_distance,hamming_distance,jaro_distance,jaro_winkler,levenshtein_distance,match_rating_comparison
0,6.0,13.0,0.665476,0.665476,6.0,True
1,6.0,13.0,0.665476,0.665476,6.0,True
100,7.0,14.0,0.794102,0.814692,7.0,True
101,7.0,14.0,0.794102,0.814692,7.0,True
102,7.0,13.0,0.590476,0.590476,7.0,True
103,17.0,22.0,0.633446,0.633446,17.0,False
104,6.0,13.0,0.665476,0.665476,6.0,True
105,6.0,13.0,0.665476,0.665476,6.0,True
106,6.0,13.0,0.665476,0.665476,6.0,True
107,7.0,14.0,0.794102,0.814692,7.0,True


In [101]:
>>> import numpy as np
>>> from sklearn.metrics import jaccard_similarity_score

In [103]:
>>> y_pred = [0, 2, 1, 3]
>>> y_true = [0, 1, 2, 3]
>>> jaccard_similarity_score(y_true, y_pred)
>>> jaccard_similarity_score(y_true, y_pred, normalize=False)

2